In [ ]:
!pip install pronouncing tensorflow  openpyxl

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 22.2 MB/s eta 0:00:00
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6233 sha256=ba9834e29d06ccfcea0aabafb6dceef59af12d3521371e3d292f1740cfdcf285
  Stored in directory: /root/.cache/pip/wheels/8b/81/fd/7edbf09827c7a7e2666e870b4c5c6b46c7ebd5defa399698bd
Successfully built pronouncing


In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
# Replace with the actual path to your XML file
file_path = '/content/Poem_with_Emotions.xlsx'

df = pd.read_excel(file_path)
df.head(10)

,Poem,Emotion
0,The fire burns with the intensity of a thousan...,Anger
1,"The ocean’s waves are a steady rhythm, a pulse...",Peace
2,"My words fall into the void, unheard and unans...",Sad
3,The rhythm of your heartbeat matches my soul’s...,Love
4,"The wind tears apart the quiet, its anger palp...",Anger
5,"Fear rises like the tide, drowning all reason.",Fear
6,My shadow feels heavier on lonely days.,Sad
7,A sudden burst of cold air extinguishes the fl...,Fear
8,"The sky stretches wide, embracing the horizon ...",Joy
9,"My voice falters, unable to carry the weight o...",Sad


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import re
import random

class ImprovedPoetryGenerator:
    def __init__(self):
        self.tokenizer = Tokenizer(filters='', lower=True, oov_token='<OOV>')
        self.max_len = 5  # Fixed 5-word lines
        self.vocab_size = 10000
        self.embed_dim = 128
        self.lstm_units = 256
        self.temperature = 0.7

        self.emotion_map = {
            'anger': ['anger', 'rage'],
            'courage': ['courage', 'bravery'],
            'fear': ['fear', 'terror'],
            'joy': ['joy', 'happiness'],
            'love': ['love', 'passion'],
            'peace': ['peace', 'calm'],
            'sad': ['sadness', 'grief'],
            'surprise': ['surprise', 'wonder']
        }

    def load_dataset(self, file_path):
        try:
            df = pd.read_excel(file_path)
            df.columns = ['Poem', 'Emotion']
            df = df.dropna()

            df['Emotion'] = df['Emotion'].str.lower().str.strip()
            df['Poem'] = df['Poem'].apply(self.clean_poem)
            processed = []
            for poem, emotion in zip(df['Poem'], df['Emotion']):
                words = poem.split()
                for i in range(0, len(words), 5):
                    line = ' '.join(words[i:i+5])
                    if len(line.split()) == 5:
                        processed.append((line, emotion))

            self.poems = processed
            print(f"Loaded {len(self.poems)} 5-word lines")
            return True
        except Exception as e:
            print(f"Error loading dataset: {e}")
            return False

    def clean_poem(self, poem):
        if not isinstance(poem, str):
            return ""
        poem = re.sub(r'[^\w\s]', '', poem).strip()
        poem = re.sub(r'\s+', ' ', poem)
        return poem.lower()

    def preprocess_data(self):
        lines, emotions = zip(*self.poems)

        self.tokenizer.fit_on_texts(lines)
        sequences = self.tokenizer.texts_to_sequences(lines)
        padded_sequences = pad_sequences(sequences, maxlen=self.max_len, padding='post')
        emotion_labels = [list(self.emotion_map.keys()).index(e) for e in emotions]
        emotion_onehot = tf.keras.utils.to_categorical(emotion_labels, num_classes=len(self.emotion_map))
        targets = np.zeros_like(padded_sequences)
        targets[:, :-1] = padded_sequences[:, 1:]

        return (padded_sequences, emotion_onehot, targets)

    def build_model(self):
        text_input = Input(shape=(self.max_len,))
        embedding = Embedding(self.vocab_size, self.embed_dim)(text_input)
        emotion_input = Input(shape=(len(self.emotion_map),))
        emotion_expanded = tf.keras.layers.RepeatVector(self.max_len)(emotion_input)
        combined = tf.keras.layers.Concatenate(axis=-1)([embedding, emotion_expanded])
        lstm_out = LSTM(self.lstm_units, return_sequences=True)(combined)
        output = Dense(self.vocab_size, activation='softmax')(lstm_out)
        self.model = Model(inputs=[text_input, emotion_input], outputs=output)
        self.model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy')
        self.model.summary()

    def train(self, epochs=50, batch_size=64):
        X, e, y = self.preprocess_data()
        self.model.fit([X, e], y, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    def generate_line(self, seed_words, emotion):
        try:
            emotion_idx = list(self.emotion_map.keys()).index(emotion)
            emotion_input = tf.keras.utils.to_categorical([emotion_idx], num_classes=len(self.emotion_map))
            current_words = seed_words.copy()
            current_seq = self.tokenizer.texts_to_sequences([' '.join(current_words)])[0]
            current_seq = pad_sequences([current_seq], maxlen=self.max_len, padding='post')[0]

            while len(current_words) < 5:
                preds = self.model.predict([np.array([current_seq]), emotion_input], verbose=0)[0][-1]
                preds = np.log(np.clip(preds, 1e-10, 1.0)) / self.temperature
                exp_preds = np.exp(preds - np.max(preds))
                preds = exp_preds / np.sum(exp_preds)

                valid_indices = [i for i in range(len(preds))
                                if self.tokenizer.index_word.get(i, '') not in ['', '<OOV>']]
                valid_probs = preds[valid_indices]
                valid_probs /= np.sum(valid_probs)

                next_word_idx = np.random.choice(valid_indices, p=valid_probs)
                next_word = self.tokenizer.index_word.get(next_word_idx, '')

                if next_word:
                    current_words.append(next_word)
                    current_seq = np.roll(current_seq, -1)
                    current_seq[-1] = next_word_idx

            return ' '.join(current_words).capitalize()
        except Exception as e:
            print(f"Error generating line: {e}")
            while len(current_words) < 5:
                current_words.append(random.choice(['time', 'heart', 'light', 'dream', 'soul']))
            return ' '.join(current_words).capitalize()

    def generate_poem(self, topic, emotion, lines=4):
        """Generate a complete poem with 4 lines of 5 words each"""
        try:
            emotion = next((e for e in self.emotion_map if emotion.lower() in self.emotion_map[e]), 'joy')
            poem = []

            for i in range(lines):
                seed = topic.split()[:1] if i == 0 else []
                line = self.generate_line(seed, emotion)
                poem.append(line + ('.' if i == lines-1 else ','))

            return f"\"{topic.capitalize()}\" ({emotion.capitalize()})\n\n" + '\n'.join(poem)
        except Exception as e:
            print(f"Error generating poem: {e}")
            fallback = [
                f"Soft {topic} whispers low,",
                f"With {emotion} it does glow,",
                "A story hearts should know,",
                "Where true feelings flow."
            ]
            return f"\"{topic.capitalize()}\" ({emotion.capitalize()})\n\n" + '\n'.join(fallback)

if __name__ == "__main__":
    generator = ImprovedPoetryGenerator()

    if generator.load_dataset("Poem_with_Emotions.xlsx"):
        generator.build_model()
        generator.train(epochs=50)

        test_cases = [
            ("broken glass", "sadness"),
            ("winter wind", "fear"),
            ("brave soldier", "courage"),
            ("summer rose", "love"),
            ("wedding ring", "love"),
            ("sleeping cat", "peace"),
            ("hidden treasure", "surprise"),
            ("creaking door", "fear")
        ]

        for topic, emotion in test_cases:
            print("\n" + "="*50)
            print(f"Generating {emotion} poem about {topic}:")
            print(generator.generate_poem(topic, emotion))

Loaded 3061 5-word lines


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 5)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 8)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_19 (Embedding)  │ (None, 5, 128)         │      1,280,000 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ repeat_vector_19          │ (None, 5, 8)           │              0 │ input_layer_5[0][0]    │
│ (RepeatVector)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_29            │ (None, 5, 136)         │              0 │ embedding_19[0][0],    │
│ (Concatenate)             │                        │                │ repeat_vector_19[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_19 (LSTM)            │ (None, 5, 256)         │        402,432 │ concatenate_29[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_19 (Dense)          │ (None, 5, 10000)       │      2,570,000 │ lstm_19[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,252,432 (16.22 MB)

 Trainable params: 4,252,432 (16.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 8.6557 - val_loss: 6.0677
Epoch 2/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 5.6732 - val_loss: 5.7043
Epoch 3/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 5.2419 - val_loss: 5.3829
Epoch 4/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 4.8935 - val_loss: 5.1809
Epoch 5/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 4.6579 - val_loss: 5.0570
Epoch 6/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 4.5054 - val_loss: 4.9826
Epoch 7/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.3948 - val_loss: 4.9167
Epoch 8/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 4.2453 - val_loss: 4.8648
Epoch 9/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.1749 - val_loss: 4.8128
Epoch 10/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 4.0625 - val_loss: 4.7663
Epoch 11/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 3.9124 - val_loss: 4.7162
Epoch 12/50
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 3.8

In [ ]:
!pip install keras tensorflow --quiet
import pandas as pd
import numpy as np
import string
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping  # Added import
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku
import warnings
warnings.filterwarnings("ignore")

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii", 'ignore')
    return txt

# Load and preprocess data
df = pd.read_excel('/content/Poem_with_Emotions.xlsx')
corpus = [clean_text(x) for x in df['Poem']]

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

# Create model with integer units
model = Sequential([
    Embedding(total_words, 150, input_length=max_sequence_len-1),
    LSTM(200, return_sequences=True),
    Dropout(0.2),
    LSTM(150),
    Dense(int(total_words/2), activation='relu'),
    Dense(total_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train quietly with early stopping
early_stop = EarlyStopping(monitor='loss', patience=5)
model.fit(predictors, label, epochs=100, verbose=0, callbacks=[early_stop])

def generate_poem(seed_text, emotion=None, num_lines=4, words_per_line=8):
    poem_lines = []
    current_line = seed_text

    for _ in range(num_lines):
        for _ in range(words_per_line):
            token_list = tokenizer.texts_to_sequences([current_line])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

            # Predict with temperature
            predictions = model.predict(token_list, verbose=0)[0]
            predictions = np.asarray(predictions).astype('float64')
            predictions = np.log(predictions) / 0.7  # Temperature
            exp_preds = np.exp(predictions)
            predictions = exp_preds / np.sum(exp_preds)
            predicted_index = np.random.choice(range(len(predictions)), p=predictions)

            output_word = ""
            for word, index in tokenizer.word_index.items():
                if index == predicted_index:
                    output_word = word
                    break

            current_line += " " + output_word

        # Split into lines
        words = current_line.split()
        line = " ".join(words[-words_per_line:]).capitalize()
        poem_lines.append(line)
        current_line = words[-1]  # Start next line with last word

    return "\n".join(poem_lines)

# Test cases
test_cases = [
    ("ocean waves", "joy"),
    ("mountain peak", "courage"),
    ("broken glass", "sad"),
    ("raging storm", "anger"),
    ("wedding ring", "love"),
    ("sleeping cat", "peace"),
    ("hidden treasure", "surprise"),
    ("creaking door", "fear")
]

print("\nGenerated Poems:\n" + "="*40)
for seed, emotion in test_cases:
    print(f"\n{emotion.upper()} - '{seed}':")
    print(generate_poem(seed, emotion))
    print("-"*40)


Generated Poems:

JOY - 'ocean waves':
Its path a cauldron of seething fury full
Of a new day fills the air with
You the world seems gentler kinder more alive
Stretch longer brighter than the world tilts haunting
----------------------------------------

COURAGE - 'mountain peak':
Low mirroring the pieces of my heart from
Your arms the chaos of the world in
The face of despair i find my strength
Blooms in the rhythm of children jumping puddles
----------------------------------------

SAD - 'broken glass':
Glitters like fallen stars of rage down my
Heart beats with the rhythm of your presence
We only threatening to freeze with the whispers
Barefoot in the grass awakens my inner child
----------------------------------------

ANGER - 'raging storm':
Glides over the sparkling sea of a new
Gentle rhythm sings of a heart alive with
You even the ordinary feels extraordinary with every
Heartbeat carries your name in silent devotion a
----------------------------------------

LOVE - 'wedding

In [ ]:
!pip install keras tensorflow  # Ensure TensorFlow is installed
import pandas as pd
import numpy as np
import string
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Import from TensorFlow
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout  # Import from TensorFlow
from tensorflow.keras.preprocessing.text import Tokenizer  # Import from TensorFlow
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import tensorflow.keras.utils as ku
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# ... (rest of your code) ...

# Load your dataset
df = pd.read_excel('/content/Poem_with_Emotions.xlsx')

# Preprocess the text
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii", 'ignore')
    return txt

corpus = [clean_text(x) for x in df['Poem']]

# Tokenization
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)

# Padding sequences and prepare variables
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

# Create LSTM model
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()

    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))

    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))

    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

model = create_model(max_sequence_len, total_words)
model.summary()

# Train the model
model.fit(predictors, label, epochs=100, verbose=5)

# Function to generate poetry with emotion
def generate_poem_with_emotion(seed_text, emotion, next_words, model, max_sequence_len):
    emotion_words = {
        'Anger': ['rage', 'fire', 'burn', 'fury', 'storm'],
        'Love': ['heart', 'love', 'soul', 'embrace', 'passion'],
        'Sad': ['tears', 'lonely', 'empty', 'weep', 'shadow'],
        'Joy': ['smile', 'laugh', 'bright', 'sun', 'happy'],
        'Fear': ['dark', 'shiver', 'unknown', 'tremble', 'ghost'],
        'Surprise': ['sudden', 'unexpected', 'wonder', 'magic', 'astonish'],
        'Peace': ['calm', 'serene', 'tranquil', 'still', 'quiet'],
        'Courage': ['brave', 'strong', 'stand', 'face', 'bold']
    }

    # Add emotion-specific seed words
    if emotion in emotion_words:
        seed_text += " " + " ".join(emotion_words[emotion][:2])

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

# Generate poems with different emotions
emotions = ['Love', 'Anger', 'Sad', 'Joy', 'Fear', 'Surprise', 'Peace', 'Courage']

print("\nGenerated Poems with Emotions:")
for emotion in emotions:
    print(f"\n--- {emotion} Poem ---")
    print(generate_poem_with_emotion("the moon", emotion, 10, model, max_sequence_len))
    print(generate_poem_with_emotion("in the forest", emotion, 10, model, max_sequence_len))

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_20 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78